![Top <](./images/watsonxdata.png "watsonxdata")

# Presto Magic Commands
Presto Magic commands make it easy to run SQL in cells in your Jupyter Notebook as is, without any Python code. This notebook shows how to enable this feature for watsonx.data and provides some examples of its use.

## Import Presto Magic Commands
The SQL code tutorials for Presto rely on a Jupyter notebook extension, commonly referred to as a "magic" command. The beginning of any notebook will require the following command which will load the extension and allow the remainder of the notebook to use the `%sql` magic command. Use the Run button or Shift-Enter to run the code in the next cell.

In [ ]:
%run presto.ipynb

The `%run` command will insert the contents of the `presto.ipynb` notebook into this notebook. The `presto` file contains a number of Python commands that provide an interface between the `%sql` syntax and the actual SQL that is presented to the Presto database engine. You can edit the `presto.ipynb` notebook like any other file to see how the magic command is implemented. 

After loading the presto notebook, all sql commands will use the `%sql` magic command syntax to issue commands against the watsonx.data Presto database. The magic command code is using a combination of the prestodb driver, SQLAlchemy, DBAPI, and Pandas dataframes. Note that this version of the SQL magic command extension does not understand all of the Presto SQL dialect. It is sufficient to run the examples in the notebook however.

## An Introduction to Magic Commands
Magic commands are a feature of Jupyter notebooks. Think of these commands as special routines that take your input and execute some logic against it. There are many types of magic commands available, including ones that allow notebooks to run system commands, time statement execution and other utility functions. The `%sql` command removes the need to understand the underlying Python, Pandas, PrestoDB, DBAPI and SQLAlchemy code. 

There are two forms of `%sql` command. The single `%` version of the command requires that the entire SQL statement remain on one line. You can extend the line by using the Python continuation character at the end of each line `\` but that can become tedious on large blocks of code.

Use the single form of the `%sql` command when:
* The SQL is short
* You need to use Python variables
* You need to save the output

#### Simple SQL statement

An example of a simple SQL statement:
```sql
%sql select count(*) from tiny.customer
```
Presto SQL can have a semicolon `;` at the end of each statement. The magic command only allows multiple statements in a `%%sql` block.

```sql
%%sql
DROP TABLE IF EXISTS CUSTOMER;
CREATE TABLE CUSTOMER ....;
```

#### Continuation Character

You can also use the `\` character to extend the SQL beyond one line.
```sql
%sql select city, lastname \
       from tiny.customer \
     order by city, lastname
```

#### SQL Block

The previous statement would have be easier to read and type by using the cell version of the `%%sql` command.
```sql
%%sql
  select city, lastname
    from tiny.customer
  order by city, lastname
```
There is less chance of forgetting the `\` at the end of each line of SQL. Note that only one SQL statement is allowed in `%sql` command, but multiple SQL statements, separated with semi-colons, is permitted with the `%%sql` command.

#### Variable Substitution

If you want to substitute a Python variable into a SQL statement, you must use the single line version `%sql` and enclose the Python variable names in braces `{variable}`. The following example will search for all employees whose first name is `Fred`.
```sql
firstname = 'Fred'
%sql select * from customer where firstname={firstname}
```

#### Result Sets

If you need to save the result set into a Python variable, you must use the single line version `%sql` and use raw, text, or pandas format for the output (more on this later).
```sql
result = %sql select * from customer
```
The result set will be placed into the Python variable `result` that you can further manipulate in your program.

## SQL Magic Command Options
The `%sql` magic command provides options to modify how the output is displayed in a Jupyter notebook. There are four display options:
* `grid` - This uses the ipydatagrid widget which provides a scrollable table in a notebook cell. This is the preferred way of viewing large answer sets.
* `pandas` - Results can be displayed using Pandas dataframes. This format will print the results in a table, but the results will be truncated and the results are not scrollable.
* `text` - As the name suggests, the output will be in text only with minimal formatting.
* `raw` - The data is returned as an array of rows and columns. This is the raw data being returned by the Presto engine so this will provide you with the most flexibility in formatting the data.

Pandas dataframes support extensive graphing, analysis and data manipulation functions. This format is recommend if you plan to do any advanced work against the data.

To set the default display mode, use the following syntax:
```sql
%sql display [ grid | pandas | text | raw ]
```

The default cell size for the grid display is 150 pixels. You can reduce this value to 100 or set it as high as 500. To change the size, use the display command with a number:
```sql
%sql display 400
```

You can temporarily override the output type by using the following flags on the `%sql` command line:
* `--grid` - display as grid
* `--pandas` - display/output as pandas
* `--text` - display/output as text
* `--raw` - display/output as an array
* `--timer` - return the number of seconds it took to execute the SQL (DML Only)
* `--prototype` - turn on or off the prototype mode

## Help
If you need a quick check of the `%sql` command syntax, use the help command.
<pre>
%sql help
Presto Magic Command Format
Command Format:
    %sql command (single line)
    %%sql 
       ... command over multiple lines ...
 
Display options
    --grid      - scrollable grid
     [50-500]   - grid size
    --pandas    - pandas dataframe
    --text      - text display
    --raw       - array of rows/columns
    --timer     - return time to execute the SQL with no output
    --prototype - turn on/off the prototype mode to display Python code 
 
Connection parameters:
%%sql connect 
      userid=value password=value
      certfile=filename
      hostname=host port=port
      catalog=catalog schema=schema
 
Change catalog: %sql use catalog.schema
Change schema : %sql use schema

Force SQL Statement Type
    [dml]      - The SQL should return an answer set
    [ddl]      - This is an SQL command with no results
</pre>

## Connecting to Presto
In order to connect to the Presto engine, you will require the following information:
* Userid - Authorized userid
* Password - Password of the userid
* Hostname - The symbolic or IP address of the host
* Port - The port that the Presto engine is listening to (8443 is the default)
* Catalog - The catalog to use when connecting
* Schema - The default schema
* Certificate file - The `.crt` certificate file that is needed to secure communications to the host

The syntax for the connect command is:
```
%sql connect setting1=value1 setting2=value2 ...
```
If the settings need to spill over multiple lines, you can use the cell `%%sql` command.

```
%%sql
  connect
  setting1=value1
  setting2=value2
```

If no schema currently exists in a catalog, or you are not sure what schema you want to use, the keyword "none" can be used for the schema name. All parameters must be in the format `setting=value`. Quotes are not used around values and in some cases may cause an error. 

The next SQL statement will connect to the watsonx.data server. These values are pre-populated and you do not need to make any changes for this to work. If you were using watsonx.data on a different server you will need to update these settings.

In [ ]:
%%sql
   connect
   userid=ibmlhadmin
   password=password
   hostname=watsonxdata
   port=8443
   catalog=tpch
   schema=tiny
   certfile=/certs/lh-ssl-ts.crt

## Setting Catalog and Schemas
The USE command is used to change to a different catalog and schema. The syntax is:
```sql
%sql use catalog.schema
```
To change only the current schema, supply the schema name to the command.
```sql
%sql use newworkshop
```
If you want to change the catalog and the schema, use the `catalog.schema` syntax:
```sql
%sql use iceberg_data.newworkshop
```
If you want to change the catalog, but do not know what schema to use (or one may not exist), use `none` as the schema name.
```sql
%sql use iceberg_data.none
```

## SQL Support
The Presto magic command determines whether your SQL statement is DDL (CREATE, ALTER, DROP, INSERT, ...) or DML (SELECT, SHOW, VALUES, ...). If the program can't determine what type of statement it is, it attempts to guess the results by checking if any output is produced. If you come across SQL that should be returning a result set, but the magic command is not returning any values, use the `[dml]` (Data Manipulate Language) keyword at the front of your SQL. If the SQL should be treated as a command, use the `[ddl]` (Data Definition Language) at the front of the SQL. The reason that these aren't flags (like `--pandas`) is that you may have multiple statements in a SQL block and only one needs to be treated differently.
```sql
%sql [dml] values 1,2,3,4
```

In [ ]:
%sql [dml] values 1,2,3,4

## SQL Magic Command Examples
A magic command starts with the keyword `%sql` or `%%sql`. When using `%sql`, the text after the keyword is sent to the Presto engine for execution. The semi-colon is not recognized as a separator for statements. If you have a SQL statement that is large and spans many lines, you can use the `%%sql` version of the command. When using `%%sql`, the text after the keyword is ignored. The lines after the `%%sql` are considered to be the SQL and are passed to the Presto engine. You can have multiple SQL statements in the text and a semicolon will be the statement terminator.

The following is an example of a single line `%sql` statement.

In [ ]:
%sql values 1,2,3,4

The previous statement displayed the results in a grid format. If you want to use Pandas format, use the `--pandas` option.

In [ ]:
%sql --pandas values 1,2,3,4

You can save the results of a `--pandas` or `--raw` format to a Python variable.

In [ ]:
x = %sql --pandas values 1,2,3,4

The variable `x` now contains the results as a Pandas dataframe which you can manipulate.

In [ ]:
x

The raw format only provides the data in rows and columns without a header.

In [ ]:
x = %sql --raw values 1,2,3,4
x

### Multiline SQL
You can spread SQL out over multiple lines instead of using continuation characters. The following two SQL blocks are identical.

In [ ]:
%sql values 1,\
            2,\
            3,\
            4

In [ ]:
%%sql
values
     1, 
     2,
     3,
     4

### Variable Substitution
You must use the single line `%sql` format to use Python variables in your SQL. Here is an example of using Python substitution in a SQL statement.

In [ ]:
values = 587
%sql values {values}

## Overview of Schemas and Catalogs

A watsonx.data catalog contains schemas and references a data source via a connector. A connector is like a driver for a database. The following commands are used to view the catalogs and schemas that are available.

Display the catalogs.

In [ ]:
%sql show catalogs

Let's look up what schemas are available with any given catalog. We will use the TPCH catalog which is an internal PrestoDB auto-generated catalog and look at the available schemas.

In [ ]:
%sql show schemas in tpch

You can connect to a specific catalog and schema and look at the tables. This requires that you update the catalog and schema that you want to view. The `%sql` driver uses the `USE` command:
```
%sql USE [schema_name]
%sql USE [catalog_name].[schema_name]
```
Connect to the `tpch` catalog and set the schema to `tiny`.

In [ ]:
%sql use tpch.tiny

Look at the available tables in the `TPCH` catalog under the `tiny` schema.

In [ ]:
%sql show tables

Inspect schema of the customer table.

In [ ]:
%sql describe customer

You could also use the syntax below to achieve the same result.

In [ ]:
%sql show columns from customer

Inspect available functions.

In [ ]:
%sql show functions like 'date%'

Switch to a different schema. The `USE [schema]` command is used to switch to a different schema. 

In [ ]:
%sql use sf1

Display the Tables in the schema.

In [ ]:
%sql show tables

Query data from customer table.

In [ ]:
%sql select * from customer limit 5

Gather statistics on a given table.

In [ ]:
%sql show stats for customer

## Prototype Setting
The following command will turn on the prototype mode for the Presto magic commands.
```sql
%sql --prototype
```
This command is used by itself and any additional text will be ignored. This setting will toggle the prototype switch in the program between on and off. When the setting is on, the Presto magic command will print the Python statements that were executed when using the magic command format. This setting is useful when you want to develop code that does not rely on the magic commands and want to use native Python calls.

The next command will turn on prototype mode.

In [ ]:
%sql --prototype

The code that was generated in the previous command will need to be used in a Python program to make sure the proper libraries are available to connect to the Presto (watsonx.data) database. The next block will run this code.

In [ ]:
# All Python code will be displayed during execution
# Imports required to connect to watsonx.data
import prestodb
from prestodb import transaction
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import warnings
warnings.filterwarnings("ignore")

The next statement will connect to the database and generate the connection code.

In [ ]:
%%sql
   connect
   userid=ibmlhadmin
   password=password
   hostname=watsonxdata
   port=8443
   catalog=tpch
   schema=tiny
   certfile=/certs/lh-ssl-ts.crt

The generated connect statement is shown below.

In [ ]:
# Connection Parameters
userid     = "ibmlhadmin"
password   = "password"
hostname   = "watsonxdata"
port       = "8443"
catalog    = "tpch"
schema     = "tiny"
certfile   = "/certs/lh-ssl-ts.crt"

# Connect Statement
try:
    connection = prestodb.dbapi.connect(
            host=hostname,
            port=port,
            user=userid,
            catalog=catalog,
            schema=schema,
            http_scheme='https',
            auth=prestodb.auth.BasicAuthentication(userid, password)
    )
    connection._http_session.verify = certfile
    cursor = connection.cursor()
    print("Connection successful")
except Exception as e:
    print("Unable to connect to the database.")
    print(repr(e))

The final step will issue a SELECT statement to return some data into a variable.

In [ ]:
%sql select * from customer limit 5

The generated code is found in the next cell.

In [ ]:
# SQL
sql = '''
select * from customer limit 5 
'''

# Executing SQL Statement (Returning Dataframe)
# Variable df contains the answer set in a Pandas dataframe
try:
    df = pd.read_sql(sql,connection)
    if (len(df) == 0):
        print("No rows found.")
except Exception as e:
    print(repr(e))

The result set is placed into the variable `df`. The next line will display the results.

In [ ]:
df

If you prefer to have the results in a Python array, you can use the `--raw` flag. Note that you have to use an assignment statement to place the results into a variable, otherwise the values will be printed out.

In [ ]:
results = %sql --raw select * from customer limit 5

The code is executed below. Note that the variable `rows` will contain the result set because the magic command does not know what variable it is returning the result set to. 

In [ ]:
# SQL
sql = '''
 select * from customer limit 5 
'''

# Executing SQL Statement (Returning Array)
# Variable rows contains an array of values from the answer set
try:
    cursor.execute(sql)
    rows = cursor.fetchall()
except Exception as e:
    print(repr(e))

To view the results, print the contents of the `rows` variable.

In [ ]:
rows

To turn off prototype mode, run the `--prototype` command again.

In [ ]:
%sql --prototype

#### Credits: IBM 2024, George Baklarz [baklarz@ca.ibm.com]